In [40]:
pip install pydicom

     |████████████████████████████████| 2.0 MB 4.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [41]:
import pandas as pd

import glob
import pydicom

In [86]:
exp1 = pd.read_csv('labels_exp1.csv')
exp2 = pd.read_csv('labels_exp2.csv')

In [87]:
exp1

,type,uuid,slice,x,y
0,FB,1280,100,323,381
1,FB,1009,76,354,288
2,FB,7794,79,163,191
3,FB,7794,94,338,243
4,FB,2997,45,156,135
...,...,...,...,...,...
128,TM,7507,201,316,362
129,TM,7507,319,125,323
130,TM,2190,48,147,379
131,TM,2190,213,141,336


In [174]:
df = exp1.append(exp2)

In [175]:
len(df.uuid.unique())

98

In [176]:
df = df.drop_duplicates(subset = 'uuid')
df = df[['uuid','type']]
df.set_index('uuid', inplace = True)
df.head(5)

,type
uuid,
1280,FB
1009,FB
7794,FB
2997,FB
2043,FB


In [177]:
len(df)

98

## Import dcm data

In [122]:
pixel_data = []

In [123]:
paths = glob.glob("/Users/yuxuanmei/Downloads/data/Tampered Scans/colletion/*/*.dcm")
for path in paths:
    dataset = pydicom.dcmread(path)
    pixel_data.append(dataset.pixel_array)

In [124]:
len(pixel_data)

22213

In [125]:
uuid = []
for path in paths:
    uuid.append(path[57:61])
len(uuid)

22213

In [126]:
dict = {'uuid':uuid, 'pixel':pixel_data}

In [131]:
total_df = pd.DataFrame(dict)
total_df.set_index('uuid', inplace=True)
total_df.head(5)

,pixel
uuid,
2244,"[[-1000, -1000, -1000, -944, -973, -1000, -975..."
2244,"[[-1000, -1000, -1000, -993, -977, -964, -992,..."
2244,"[[-1000, -1000, -1000, -1000, -1000, -985, -97..."
2244,"[[-1000, -1000, -940, -984, -999, -981, -1000,..."
2244,"[[-1000, -1000, -979, -964, -997, -990, -990, ..."


In [181]:
label_dict = df['type'].to_dict()
len(label_dict)

98

In [148]:
total_df.reset_index(inplace=True)

In [190]:
total_df['label'] = ''

In [201]:
for i in range(len(total_df)):
    total_df.iloc[i,2] = label_dict[int(total_df.iloc[i,0])]

In [206]:
total_df

,uuid,pixel,label
0,2244,"[[-1000, -1000, -1000, -944, -973, -1000, -975...",TM
1,2244,"[[-1000, -1000, -1000, -993, -977, -964, -992,...",TM
2,2244,"[[-1000, -1000, -1000, -1000, -1000, -985, -97...",TM
3,2244,"[[-1000, -1000, -940, -984, -999, -981, -1000,...",TM
4,2244,"[[-1000, -1000, -979, -964, -997, -990, -990, ...",TM
...,...,...,...
22208,2838,"[[-3024, -3024, -3024, -3024, -3024, -3024, -3...",FM
22209,2838,"[[-3024, -3024, -3024, -3024, -3024, -3024, -3...",FM
22210,2838,"[[-3024, -3024, -3024, -3024, -3024, -3024, -3...",FM
22211,2838,"[[-3024, -3024, -3024, -3024, -3024, -3024, -3...",FM


In [215]:
total_df.pixel[0].shape

(512, 512)

In [216]:
total_df.to_pickle('cleaned_data.pkl')

In [218]:
test = pd.read_pickle('cleaned_data.pkl')

In [220]:
test.pixel[0].shape

(512, 512)